In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import yfinance as yf

In [2]:
yf.pdr_override()
tickers = ['AAPL', 'ABC', 'ABT', 'ADBE', 'ADM', 'ADP', 'ADSK', 'AEP', 'AIR',
       'AMD', 'AMGN', 'ATVI', 'AXP', 'AZN', 'BA', 'BAX', 'BCE', 'BMY',
       'BSX', 'BTI', 'C', 'CAT', 'CHD', 'CL', 'CLX', 'CMCSA', 'COST',
       'CSCO', 'CVS', 'DD', 'DE', 'DIS', 'DTE', 'DUK', 'DVN', 'EA', 'ED',
       'EFX', 'EMR', 'ETN', 'F', 'FCX', 'FDX', 'FISV', 'GD', 'GE', 'GIS',
       'HAL', 'HD', 'HOG', 'HON', 'HRL', 'HSIC', 'HTHIY', 'HUM', 'IBM',
       'INTC', 'INTU', 'IP', 'ITW', 'JNJ', 'JPM', 'K', 'KO', 'LLY', 'LMT',
       'LUV', 'MCD', 'MDT', 'MMM', 'MO', 'MRK', 'MSFT', 'NKE', 'NVO',
       'NYT', 'OMC', 'ORCL', 'PCRFY', 'PEP', 'PFE', 'PG', 'PHG', 'RELX',
       'RTX', 'SAP', 'SBUX', 'SCHW', 'SLB', 'SO', 'SONY', 'T', 'TJX',
       'TM', 'TRV', 'TXN', 'UL', 'VOD', 'VZ', 'WBA', 'WHR', 'WM', 'WMT',
       'WPP', 'XOM']
start = '1997-01-01'
end = '2021-12-31'

# DataFrame mit den Adj Close Preisen der Ticker erstellen
Adj_Close = web.get_data_yahoo(tickers, start, end)['Adj Close']
Adj_Close = pd.DataFrame(Adj_Close)

# Anzahl fehlender Werte sollte 0 sein
print(Adj_Close.isna().sum().sum())

# Logreturns berechnen
returns = np.log(Adj_Close/Adj_Close.shift(1))
returns = returns.dropna()

[*********************100%***********************]  105 of 105 completed
0


In [3]:
# Dataframe kürzen 1997-2001
insample = returns.loc['1997-01-01':'2001-12-31']

# Varianz-Covarianz-Matrix berechnen
cov_matrix = insample.cov()

# Varianzen löschen (Hauptdiagonale = 0)
for i in range(len(cov_matrix)):
    cov_matrix.iloc[i,i] = 0

In [4]:
# Minimierung der Summe der Covarianzen im Portfolio
# Funktion 1
def calc_min_covariance(selected_assets, num_assets, cov_matrix):
    covariances = selected_assets * cov_matrix #slected_assets als freie Variable (alle möglichen 15-Asset-Portfolios) * Covarianz-Matrix aller Assets -> Covarianz-Matrix des aktuell betrachteten Portfolios
    portfolio_covariance = np.sum(np.sum(covariances)) #Summe der Portfolio-Covarianz
    return portfolio_covariance

# Funktion 2
def min_port_covariance(num_assets, cov_matrix):
    args = (num_assets, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - num_assets})
    start = np.concatenate((np.ones(num_assets), np.zeros(len(tickers) - num_assets)), axis=0)
    bound = (0, 1)
    bounds = tuple(bound for asset in range(len(tickers)))
    result = sco.minimize(calc_min_covariance, start, args=args, 
                          method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Ausführen der Funktionen und runden der Gewichtungen auf 0 bzw. 1
selected_assets = np.round(min_port_covariance(15, cov_matrix))
selected_assets

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.])

In [5]:
# Min Covariance Portfolio aus allen Tickern extrahieren
tickers_np = np.array(tickers)
portfolio_tickers = tickers_np[selected_assets==1].tolist()
print(portfolio_tickers)
len(portfolio_tickers)

['AEP', 'AZN', 'BTI', 'DTE', 'DUK', 'DVN', 'ED', 'FCX', 'GIS', 'HRL', 'LMT', 'NVO', 'RELX', 'SO', 'XOM']


15

In [18]:
# Summe der Covariancen des Portfolios berechnen und mit Beispielportfolios vergleichen
#portfolio_returns = insample[0:15] #ersten 15 Aktien
#portfolio_returns = insample[40:55]
#portfolio_returns = insample[90:105]
portfolio_returns = insample[portfolio_tickers] #Min Covariance Portfolio
portfolio_cov = portfolio_returns.cov()

# Varianzen löschen (Hauptdiagonale = 0)
for i in range(len(portfolio_cov)):
    portfolio_cov.iloc[i,i] = 0

# Summe der Covarianzen im Portfolio
portfolio_covariance = np.sum(np.sum(portfolio_cov))
print(portfolio_covariance)

0.01031381084454732
